# Python Homework 1: Alignment


### Evaluation

To check the correctness of your code, we will run it against prepared test cases.


### Notes

* Always restart the notebook and rerun from the top to double-check (i.e. reproduce) your solutions.
* Here we use the terms "cost", "distance", and "penalty" interchangeably since in this context they represent equivalent ways of looking at the same concepts. It also helps illustrate that these algorithms are quite general and can be applied to many different problems.

## Global Alignment 

### Assignment 1.1.

Fill in the "blanks" in the code below (replace the `# WRITE CONDITION HERE`) with the appopriate logical checks
such that this distance function returns the correct distance between two letters.

Remember to evaluate the code cell so that the notebook starts using your function.
(To evaluate, press `[Shift] + [Enter]` when the edit cursor is in the cell).

In [2]:
def dist_global(char1, char2):
    """
    Function that computes the distance between two characters in a sequence,
    used for global alignment.
    
    # Input arguments:
    char1, char2 = Values of 2 characters, either nucleobases or gaps, as 1-character strings.
    
    E.g. char1 = 'A', char2 = 'G'
         char1 = 'A', char2 = '-'

    # Returns:
    The distance (or cost of change) between char1 and char2, using the DIST_... values
    """
    
    DIST_MATCH = 0
    DIST_GAP = 8
    DIST_TRANSITION = 2
    DIST_TRANSVERSION = 4
    
    if char1 == char2:
        return DIST_MATCH

    if char1== '-' or char2 == '-':
        return DIST_GAP

    if char1== ('A' or 'G') and char2 == ('G' or 'A'):
        return DIST_TRANSITION

    if char1==('C' or 'T') and char2 == ('T' or 'C'):
        return DIST_TRANSITION

    else:
        return DIST_TRANSVERSION

Check your code with the test below, assuming you use the same values as in the lecture. If you want to experiment with different values, then naturally the test will have to be adapted.

It's a good idea to write simple tests for your code even before you actually write the code, if possible.
That way you make sure you actually understand what you're solving
and provide a sanity check if you need to change the code but keep the same overall logic.

#### Test

In [3]:
print(dist_global('A', 'G') == 2)
print(dist_global('A', '-') == 8)
print(dist_global('-','-') == 0)
print(dist_global('-','G') == 8)
print(dist_global('C','A') == 4) # transversion works

True
True
True
True
True


### Needleman–Wunsch algorithm

Here you are provided with skeleton code to compute the global alignment 
between two sequences using the Needleman–Wunsch algorithm algorithm.

### Assignment 1.2.

Fill in the "blanks" in the code (replace `### WRITE CODE HERE ###` with actual code)
and thus finish this implementation of the algorithm.

*Check your code* with the test below this cell.
This code cell requires `dist_global()` from the previous assignment, so make sure
to solve that first and evaluate that cell (`[Shift]` + `[Enter]`) so that the notebook is aware of it.

In [16]:
import numpy as np

def global_alignment_matrix(seq1, seq2):
    """
    Computes the global alignment matrix D between 
    sequences seq1 and seq2
    
    Input:
    seq1, seq2 = nucleobase sequences as strings
    
    E.g. seq1 = 'TATACCTGAAGGGCCT', seq2 = 'TATACGAGACCGTTT'
    
    Returns:
    D = global alignment matrix as 2D numpy array
    """
    #Match_pen=0
    #Gap_pen=8
    #Transversion_pen=4
    #Transition_pen= 2
    # Initialize the matrix with all zero values
    # (and enforce use of integers, since numpy works with floats by default)
    D = np.zeros((len(seq1) + 1, len(seq2) + 1), dtype=int) 
    
    # Initialize horizontal (top) boundary of D using seq1
    for i in range(1, len(seq1) + 1):
        D[i, 0] = 8*i     # I presume we consider it as gaps, so we add 8
        
    # Initialize vertical (left) boundary of D using seq2
    for j in range(1, len(seq2) + 1):
        D[0, j] = 8*j ### WRITE CODE HERE ###
    
    # Walk through all cells of the matrix,
    # updating the current cell by considering a step from 3 possible directions,
    # and taking the least "costly" of these 3 directions (shortest distance).
    for i in range(1, len(seq1) + 1): # i is going across all rows
        for j in range(1, len(seq2) + 1): # j is going across all columns
            
            dist_from_upper_left = dist_global(seq1[i-1], seq2[j-1])
            dist_from_above      = dist_global(seq1[i-1], '-')
            dist_from_left       = dist_global('-',       seq2[j-1])
            
            D[i, j] = min(D[i-1, j-1] + dist_from_upper_left,
                          D[i-1, j  ] + dist_from_left,
                          D[i  , j-1] + dist_from_above)
    return D


def global_alignment_value(global_alignment_matrix):
    """
    Input:
    global_alignment_matrix = a computed global alignment matrix as a numpy array
    
    Returns:
    The lower-right corner of the matrix, since we know that holds
    the global alignment value
    """
    lower_right_corner = (global_alignment_matrix.shape[0] - 1, # gives index of last row
                          global_alignment_matrix.shape[1] - 1) # gives index of last column
    return global_alignment_matrix[lower_right_corner]

#### Test

In [17]:
seq1 = 'TACGTCAGC'
seq2 = 'TATGTCATGC'

D = global_alignment_matrix(seq1, seq2)

print(D)
print()
print(global_alignment_value(D))



[[ 0  8 16 24 32 40 48 56 64 72 80]
 [ 8  0  8 16 24 32 40 48 56 64 72]
 [16  8  0  8 16 24 32 40 48 56 64]
 [24 16  8  2 10 18 24 32 40 48 56]
 [32 24 16 10  2 10 18 26 34 40 48]
 [40 32 24 16 10  2 10 18 26 34 42]
 [48 40 32 24 18 10  2 10 18 26 34]
 [56 48 40 32 26 18 10  2 10 18 26]
 [64 56 48 40 32 26 18 10  6 10 18]
 [72 64 56 48 40 34 26 18 12 10 10]]

10


**Expected Output**

```
[[ 0  8 16 24 32 40 48 56 64 72 80]
 [ 8  0  8 16 24 32 40 48 56 64 72]
 [16  8  0  8 16 24 32 40 48 56 64]
 [24 16  8  2 10 18 24 32 40 48 56]
 [32 24 16 10  2 10 18 26 34 40 48]
 [40 32 24 16 10  2 10 18 26 34 42]
 [48 40 32 24 18 10  2 10 18 26 34]
 [56 48 40 32 26 18 10  2 10 18 26]
 [64 56 48 40 32 26 18 10  6 10 18]
 [72 64 56 48 40 34 26 18 12 10 10]]

10
```

## Local Alignment

### Assignment 1.3.

Similarly to Assignment 1.1., fill in the blanks in the Python code and finish the function that gives the distance between two characters.

In [18]:
def dist_local_example(char1, char2):
    """
    Function that computes the distance between two characters in a sequence,
    used for local alignment.
    
    # Input arguments:
    char1, char2 = Values of 2 characters, either nucleobases or gaps, as 1-character strings.
    
    E.g. char1 = 'A', char2 = 'G'
         char1 = 'A', char2 = '-'

    # Returns:
    The distance (or cost of change) between char1 and char2, using the DIST_... values
    """
    
    DIST_MATCH = 2
    DIST_GAP = -6
    DIST_MISMATCH = -4
    
    if char1==char2:
        return DIST_MATCH
    
    if char1 == '-' or char2 == '-':
        return DIST_GAP
    
    else:
        return DIST_MISMATCH

Check that the function is correct using the code test below (assuming you use the same costs values as in the lecture).
#### Test

In [19]:
print(dist_local_example('A', '-') + dist_local_example('A', 'T') == -10)
print(dist_local_example('A', 'C') + dist_local_example('G', 'T') == -8)
print(dist_local_example('C', 'C') + dist_local_example('-', 'T') == -4)

True
True
True


### Smith-Waterman Local Alignment Algorithm

Since it is a variation of the global alignment algorithm,
the Smith-Waterman algorithm has a similar structure.
A big difference is that one takes the maximum of different possibilities,
since once keeps track of an optimum score.

### Assignment 1.4.

Similar to Assignment 1.2 Your task is to fill in the "blanks" and in the ocode and write the Python code that implements it.

In [20]:
def local_alignment_matrix(seq1, seq2):
    """
    Computes the local alignment matrix D between 
    sequences seq1 and seq2
    
    Input:
    seq1, seq2 = nucleobase sequences as strings
    
    E.g. seq1 = 'TATACCTGA', seq2 = 'TATACGAGACCGTTT'
    
    Returns:
    D = local alignment matrix as 2D numpy array
    """

    # Initialize the matrix with all zero values
    D = np.zeros((len(seq1) + 1, len(seq2) + 1), dtype=int)
    
    # Walk through all cells of the matrix,
    # updating the current cell by considering a step from 3 possible directions,
    # and taking the max of scores corresponding 3 directions.
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            
            dist_from_upper_left = dist_local_example(seq1[i-1],seq2[j-1])
            dist_from_above = dist_local_example(seq1[i-1], '-')
            dist_from_left = dist_local_example('-', seq2[j-1])
            
            D[i, j] = max(D[i-1,j-1] + dist_from_upper_left, # coming from diagonal
                          D[i-1,j] + dist_from_above, # i denotes a row. so it is the same column but different row
                          D[i,j-1] + dist_from_left,    # j denotes cloumn,
                          0)  # make sure to have a non-negative value since this is a distance
    return D


def local_alignment_value(alignment_matrix):
    """
    Input:
    global_alignment_matrix = a computed local alignment matrix as a numpy array
    
    Returns:
    The max value of the matrix since we know this 
    """
    return alignment_matrix.max()

#### Test

**Expected Output**

```
array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  2,  0,  2,  2,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  2,  0,  2,  4,  0,  2,  0,  0,  0],
       [ 0,  2,  0,  2,  0,  2,  0,  0,  0,  0,  0,  0,  4,  2,  2],
       [ 0,  0,  4,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  2,  0,  6,  0,  6,  0,  0,  0,  0,  0,  0,  2,  2,  2],
       [ 0,  0,  0,  0,  2,  0,  8,  2,  2,  2,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  2, 10,  4,  0,  4,  0,  0,  0,  0],
       [ 0,  2,  0,  2,  0,  2,  0,  4,  6,  0,  0,  0,  2,  2,  2],
       [ 0,  0,  0,  0,  0,  0,  4,  0,  6,  8,  2,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  2,  0,  2,  8,  4,  4,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  4,  0,  2, 10,  4,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  2,  0,  6,  2,  4, 12,  6,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  4,  0,  2,  4,  6,  8,  2,  0],
       [ 0,  2,  0,  2,  0,  2,  0,  0,  0,  0,  0,  0,  8, 10,  4],
       [ 0,  0,  4,  0,  4,  0,  0,  0,  0,  0,  0,  0,  2,  4,  6]])

12
```

Since it may be difficult to quickly check large matrices by eye,
we can use the `numpy.array_equal()` function to check if the local alignment matrix
returned by your code matches the expected matrix.

In [21]:
seq1 = 'GGTATGCTGGCGCTA'
seq2 = 'TATATGCGGCGTTT'

D = local_alignment_matrix(seq1, seq2)
expected_matrix = np.array(
   [[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
   [ 0,  0,  0,  0,  0,  0,  2,  0,  2,  2,  0,  2,  0,  0,  0],
   [ 0,  0,  0,  0,  0,  0,  2,  0,  2,  4,  0,  2,  0,  0,  0],
   [ 0,  2,  0,  2,  0,  2,  0,  0,  0,  0,  0,  0,  4,  2,  2],
   [ 0,  0,  4,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
   [ 0,  2,  0,  6,  0,  6,  0,  0,  0,  0,  0,  0,  2,  2,  2],
   [ 0,  0,  0,  0,  2,  0,  8,  2,  2,  2,  0,  2,  0,  0,  0],
   [ 0,  0,  0,  0,  0,  0,  2, 10,  4,  0,  4,  0,  0,  0,  0],
   [ 0,  2,  0,  2,  0,  2,  0,  4,  6,  0,  0,  0,  2,  2,  2],
   [ 0,  0,  0,  0,  0,  0,  4,  0,  6,  8,  2,  2,  0,  0,  0],
   [ 0,  0,  0,  0,  0,  0,  2,  0,  2,  8,  4,  4,  0,  0,  0],
   [ 0,  0,  0,  0,  0,  0,  0,  4,  0,  2, 10,  4,  0,  0,  0],
   [ 0,  0,  0,  0,  0,  0,  2,  0,  6,  2,  4, 12,  6,  0,  0],
   [ 0,  0,  0,  0,  0,  0,  0,  4,  0,  2,  4,  6,  8,  2,  0],
   [ 0,  2,  0,  2,  0,  2,  0,  0,  0,  0,  0,  0,  8, 10,  4],
   [ 0,  0,  4,  0,  4,  0,  0,  0,  0,  0,  0,  0,  2,  4,  6]])


print("Alignment matrix matches expected matrix:", np.array_equal(expected_matrix, D))
print(D)
print()
print(local_alignment_value(D))

Alignment matrix matches expected matrix: True
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  2  2  0  2  0  0  0]
 [ 0  0  0  0  0  0  2  0  2  4  0  2  0  0  0]
 [ 0  2  0  2  0  2  0  0  0  0  0  0  4  2  2]
 [ 0  0  4  0  4  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  6  0  6  0  0  0  0  0  0  2  2  2]
 [ 0  0  0  0  2  0  8  2  2  2  0  2  0  0  0]
 [ 0  0  0  0  0  0  2 10  4  0  4  0  0  0  0]
 [ 0  2  0  2  0  2  0  4  6  0  0  0  2  2  2]
 [ 0  0  0  0  0  0  4  0  6  8  2  2  0  0  0]
 [ 0  0  0  0  0  0  2  0  2  8  4  4  0  0  0]
 [ 0  0  0  0  0  0  0  4  0  2 10  4  0  0  0]
 [ 0  0  0  0  0  0  2  0  6  2  4 12  6  0  0]
 [ 0  0  0  0  0  0  0  4  0  2  4  6  8  2  0]
 [ 0  2  0  2  0  2  0  0  0  0  0  0  8 10  4]
 [ 0  0  4  0  4  0  0  0  0  0  0  0  2  4  6]]

12
